<a href="https://colab.research.google.com/github/Satyadeep-Dey/AI-experiments/blob/main/3_OpenAI_%2B_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install openai --quiet

In [ ]:
# Import required libraries

import requests
import json
from openai import OpenAI
from google.colab import userdata

#define constants
MODEL = "gpt-4o-mini"

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
print(openai_api_key)
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Check if Open AI key has been set
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
#Try a simple prompt

system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("what is the capital of Meghalaya ? ")

# Calling functions using tools

In [ ]:
# function that returns weather data from open-meteo.com API

def get_weatherV2(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


In [ ]:
# you can call this tools , abc , .. or anything you like ..
# remember to use the same name when you call Open AI .
# this name will map to tools parameter

tools = [{
    "type": "function",
    "function": {
        "name": "get_weatherV2",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
def ask_question(question):

    messages = [{'role': 'user', 'content': question}]

    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,       # this "tools" has been defined in previous cell of type function ...
        tool_choice="auto" # Model decides when to call the tool
    )

    if completion.choices[0].finish_reason=="tool_calls":
        # If tool is called
        tool_call = completion.choices[0].message.tool_calls[0]

        #print(tool_call ) #-> tool_call has function name and value of it's arguments
        #Open AI looks at the tools definiton
        #understands it needs properties latitude and longitude and GETS THIS DATA FOR US !!

        #we can then extract the value of these arguments
        args = json.loads(tool_call.function.arguments)

        # then we programatically call the function we have already written
        result = get_weatherV2(args["latitude"], args["longitude"])

        messages.append(completion.choices[0].message)  # append model's function call message

        messages.append({                               # append result message
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result)
            })


    completion_2 = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools)

    return completion_2.choices[0].message.content

In [ ]:
print(ask_question("What is the capital of Goa "))
print("****************************************")
print(ask_question("What's the weather like in Guwahati today?"))

In [ ]:
print(ask_question("What's the weather like in Sonipat today?"))